In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import cv2
import os

from lac.slam.feature_tracker import FeatureTracker
from lac.perception.segmentation import UnetSegmentation, SemanticClasses
from lac.slam.visual_odometry import StereoVisualOdometry
from lac.mapping.mapper import interpolate_heights
from lac.mapping.map_utils import get_geometric_score
from lac.utils.plotting import plot_poses, plot_surface, plot_3d_points
from lac.utils.geometry import crop_points
from lac.util import load_data
from lac.params import LAC_BASE_PATH, SCENE_BBOX

%load_ext autoreload
%autoreload 2

# Height mapping

Use stereo depth and wheel contact to generate height map (assuming perfect localization)


In [ ]:
# Load the data logs
data_path = "/home/shared/data_raw/LAC/runs/full_spiral_map1_preset0"
# data_path = Path(LAC_BASE_PATH) / "output/DataCollectionAgent/lander_loop_closure_teleop"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

left_path = Path(data_path) / "FrontLeft"
right_path = Path(data_path) / "FrontRight"
img_frames = sorted(int(img_name.split(".")[0]) for img_name in os.listdir(left_path))

# Load the ground truth map
map = np.load(
    "/home/shared/data_raw/LAC/heightmaps/competition/Moon_Map_01_preset_0.dat",
    allow_pickle=True,
)

# Stereo depth points


In [ ]:
segmentation = UnetSegmentation()
feature_tracker = FeatureTracker(cam_config)
svo = StereoVisualOdometry(cam_config)

In [ ]:
START_FRAME = 80
END_FRAME = img_frames[-1]

depth_points = []

for frame in tqdm(range(START_FRAME, END_FRAME, 2)):
    img_name = f"{frame:06}.png"
    left_img = cv2.imread(str(left_path / img_name), cv2.IMREAD_GRAYSCALE)
    right_img = cv2.imread(str(right_path / img_name), cv2.IMREAD_GRAYSCALE)

    # Segmentation
    left_pred = segmentation.predict(left_img)
    left_ground_mask = left_pred == SemanticClasses.GROUND.value

    # Stereo depth
    feats_left, feats_right, matches, depths = feature_tracker.process_stereo(
        left_img, right_img, return_matched_feats=True
    )
    kps_left = feats_left["keypoints"][0].cpu().numpy()
    ground_idxs = []
    for i, kp in enumerate(kps_left):
        u = int(kp[0])
        v = int(kp[1])
        if u < 0 or u >= left_img.shape[1] or v < 0 or v >= left_img.shape[0]:
            print(u, v)
            continue
        if left_ground_mask[v, u]:
            ground_idxs.append(i)
    ground_kps = kps_left[ground_idxs]
    ground_depths = depths[ground_idxs]
    ground_points_world = feature_tracker.project_stereo(poses[frame], ground_kps, ground_depths)
    depth_points.append(ground_points_world)

In [ ]:
all_depth_points = np.concatenate(depth_points, axis=0)
print(all_depth_points.shape)

In [ ]:
MAP_BBOX = np.array([[-13.5, -13.5, 0.0], [13.5, 13.5, 5.0]])
all_depth_points_cropped = crop_points(all_depth_points, MAP_BBOX)
print(all_depth_points_cropped.shape)

In [ ]:
fig = plot_surface(map)
fig = plot_3d_points(all_depth_points_cropped[::100], fig=fig)
fig.update_layout(width=1600, height=900, scene_aspectmode="data")
fig.show()

In [ ]:
fig.write_html("stereo_depth_height_points.html")

For each cell, take the median of points inside


In [ ]:
from scipy.stats import binned_statistic_2d

In [ ]:
points_to_fit = all_depth_points_cropped
x, y, z = points_to_fit[:, 0], points_to_fit[:, 1], points_to_fit[:, 2]

x_min, x_max = -13.5, 13.5
y_min, y_max = -13.5, 13.5
N = len(map[:, 0, 0])

grid_medians, x_edges, y_edges, _ = binned_statistic_2d(
    x, y, z, statistic="median", bins=N, range=[[x_min, x_max], [y_min, y_max]]
)
# Set Nans to -np.inf
grid_medians[np.isnan(grid_medians)] = -np.inf

In [ ]:
agent_map = map.copy()
agent_map[:, :, 2] = grid_medians
agent_map = interpolate_heights(agent_map)

In [ ]:
plot_surface(agent_map)

In [ ]:
from lac.utils.plotting import plot_heightmaps

plot_heightmaps(map, agent_map)

In [ ]:
fig.write_html("height_maps.html")

In [ ]:
get_geometric_score(map, agent_map)

# Fit the points


In [ ]:
from scipy.interpolate import griddata
from scipy.stats import zscore
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

In [ ]:
grid_xy = map[:, :, :2]
x_grid = grid_xy[:, :, 0].flatten()
y_grid = grid_xy[:, :, 1].flatten()

points_to_fit = all_depth_points_cropped[::100]

x_points = points_to_fit[:, 0]
y_points = points_to_fit[:, 1]
z_points = points_to_fit[:, 2]

In [ ]:
# Reshape x_points, y_points into a single input array for GP
X = np.vstack((x_points, y_points)).T  # Shape (N, 2)

# Define the kernel: RBF kernel with a constant multiplier
kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))

# Fit the GaussianProcessRegressor
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-2)
gp.fit(X, z_points)

In [ ]:
# Calculate Z-scores for the z-values (elevation)
z_scores = zscore(z_points)

# Set a threshold to define outliers (e.g., z > 3 or z < -3)
threshold = 3
inliers = np.abs(z_scores) < threshold
print(f"Number of inliers: {np.sum(inliers)} out of {len(z_scores)}")

# Filter out the outliers
x_points_clean = x_points[inliers]
y_points_clean = y_points[inliers]
z_points_clean = z_points[inliers]

In [ ]:
grid_z = griddata((x_points, y_points), z_points, (x_grid, y_grid), method="cubic")
grid_z = grid_z.reshape(grid_xy.shape[:2])

In [ ]:
# Visualize the result
plt.imshow(grid_z, extent=(min(x_grid), max(x_grid), min(y_grid), max(y_grid)), origin="lower")
plt.colorbar(label="Elevation (z)")
plt.title("Fitted Surface (Elevation Grid from grid_xy)")
plt.show()

In [ ]:
agent_map = map.copy()
agent_map[:, :, 2] = grid_z

In [ ]:
plot_surface(agent_map)